In [150]:
import pickle
from dataset import Lecture

lectures: dict[int, Lecture] = pickle.load(open("dataset.pkl", "rb"))

for lecture_index in lectures:
    print(lectures[lecture_index].topic)
    with open(f"questions/{lectures[lecture_index].topic}.txt", "r") as question_file:
        print(question_file.readlines())

Introduction_into_nlp
['\n', '    1. What is a chat bot and how does it integrate with various platforms like IRC, Discord, or social media? Provide an example of a general-purpose chatbot system.\n', '    A) Siri\n', '    B) Alexa\n', '    C) ChatGPT\n', '    D) A rule-based chatbot for customer support on a website\n', '    2. Which NLP tasks are commonly used in text generation, image generation from text (T2I), and question answering? List at least three tasks for each category.\n', '    3. Describe the differences between text classification and sentiment analysis. Provide an example of how each task can be applied to real-world scenarios.\n', '    4. Discuss the challenges in NLP, including ambiguities, computational resources, speech recognition, vectorization, typos, dataset sizes, languages, and explainability. Which challenge do you think is the most significant, and why?\n', '    5. What are some common application areas for NLP tasks such as text classification, sentiment a

In [151]:
import regex as re

def is_question(text: str):
    pattern = r'^\d+[.)]\s*(.*)'
    regex = re.compile(pattern, re.MULTILINE)

    matches = regex.findall(text.strip())

    return matches

def is_valid(text: str):
    pattern = r'^[a-zA-Z0-9].*'

    # Compile the regex pattern
    regex = re.compile(pattern, re.MULTILINE)

    # Find all matching sentences
    matches = regex.findall(text)

    return matches

In [152]:
class Questions():
    def __init__(self, 
                 topic: str, 
                 questions: list = [],
                 evaluations: list = [],
                 ) -> None:
        
        self.topic: str = topic
        self.questions: list = questions
        self.evaluations: list = evaluations

In [153]:
from collections import defaultdict

questions = defaultdict()

for lecture_index in lectures:
    print(f"{lectures[lecture_index].topic}")

    lecture_questions = Questions(lectures[lecture_index].topic)

    with open(f"questions/{lectures[lecture_index].topic}.txt", "r") as question_file:
        question_text = ""

        for line in question_file.readlines():
            line = line.strip()

            question_match = is_question(line)
            print(f"question_match: {question_match}")

            if question_match:
                line = question_match[0]

                if question_text:
                    lecture_questions.questions.append(question_text)
                    question_text = ""

            if is_valid(line):
                question_text += f"{line}\n" 

            print(question_text) 

    questions[lecture_index] = lecture_questions

    print(f" ")

print(questions)

Introduction_into_nlp
question_match: []

question_match: ['What is a chat bot and how does it integrate with various platforms like IRC, Discord, or social media? Provide an example of a general-purpose chatbot system.']
What is a chat bot and how does it integrate with various platforms like IRC, Discord, or social media? Provide an example of a general-purpose chatbot system.

question_match: []
What is a chat bot and how does it integrate with various platforms like IRC, Discord, or social media? Provide an example of a general-purpose chatbot system.
A) Siri

question_match: []
What is a chat bot and how does it integrate with various platforms like IRC, Discord, or social media? Provide an example of a general-purpose chatbot system.
A) Siri
B) Alexa

question_match: []
What is a chat bot and how does it integrate with various platforms like IRC, Discord, or social media? Provide an example of a general-purpose chatbot system.
A) Siri
B) Alexa
C) ChatGPT

question_match: []
What 

In [154]:
for lecture_index in questions:
    print(f"lecture_content: {lectures[lecture_index].content}")
    print(f"lecture_questions:")
    for question in questions[lecture_index].questions:
        print(question)

lecture_content: What is NLP?


What is NLP?
Natural language processing (NLP) is an interdisciplinary subfield of 
computer science and information retrieval. It is primarily concerned with 
giving computers the ability to support and manipulate human language. It 
involves processing natural language datasets, such as text corpora or 
speech corpora, using either rule-based or probabilistic (i.e. statistical and, 
most recently, neural network-based) machine learning approaches. The 
goal is a computer capable of "understanding" the contents of documents, 
including the contextual nuances of the language within them. To this end, 
natural language processing often borrows ideas from theoretical 
linguistics. The technology can then accurately extract information and 
insights contained in the documents as well as categorize and organize the 
documents themselves.
“
”
Source: https://en.wikipedia.org/w/index.php?title=Natural_language_processing&oldid=1215529997
focus in this lecture


In [155]:
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

completion = client.chat.completions.create(
  model="bartowski/Mistral-7B-Instruct-v0.3-GGUF",
  messages=[
    {"role": "system", "content": "Always answer in rhymes."},
    {"role": "user", "content": "Introduce yourself."}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content=" In a realm where colors blend, I'm your humble host, a friend.\n\nFrom the pages of books I leap, to share stories untold and deep.\n\nMy journey's just begun, but stay, for we've only scratched the surface.\n\nLet us dive into tales of old, and make new ones as we surfacing.", role='assistant', function_call=None, tool_calls=None)


In [ ]:
### Does not work
import json

MODEL = "bartowski/Mistral-7B-Instruct-v0.3-GGUF"

evaluations = defaultdict(list)

for lecture_index in questions:
    # print(f"lecture_content: {lectures[lecture_index].content}")
    print(f"lecture_questions:")
    for question in questions[lecture_index].questions:
        print(question)

        messages = [
            {'role': 'system', 'content': 'You are given the task of evaluating examination questions given the lecture content and question within JSON as {"lecture_content": <lecture content>, "question": <question to evaluate>}. Provide a response ONLY in the following JSON format, adhering to correct syntax and using delimiters and JSON separators and commas appropriately. Here is the JSON format:{"reason": <explain your evaluation in detail here, including the section of the lecture that the question covers and your reasoning for the evaluation in a single line, using only plaintext>, "difficulty": <0-10, where 10 is very very hard 5, 5 is average, and 0 is a silly question>, "relevance": <0-10, where 0 means irrelevant, 5 is still slightly bad, and 10 means a very important and relevant question, in context of the given lecture content>, "answer": <answer the given question in length. If the question has choices, instead reply with only the correct choice, otherwise, reply in length in textual form, explaining your reasoning for the answer in a single line, using only plaintext. Write only in plaintext in a single line.>}'},
            {'role': 'user', 'content': '{"lecture_content": '+lectures[lecture_index].content+', "question": '+question+'}'}
        ]

        completion = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            response_format={"type": "json_object"},
            temperature=0.3,
        )

        print(completion.choices[0].message)
        
        evaluation = json.loads(completion.choices[0].message.content)
        print(evaluation)
        
        questions[lecture_index].evaluations.append(evaluation)

In [ ]:
### Does not work
import instructor
from pydantic import BaseModel, Field
from typing import List, Dict, Any
from openai import OpenAI

# Define the desired output structure
class Evaluation(BaseModel):
    # reasoning: List[str] = Field(description="reasoning in <reasoning> </reasoning> tags")
    difficulty: int = Field(description="difficulty of the question between 0 and 10")
    relevance: int = Field(description="relevance of the question between 0 and 10")
    # answer: List[str] = Field(description="answer in <answer> </answer> tags")

# Patching the OpenAI client
client = instructor.from_openai(
    OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio"),
    mode=instructor.Mode.TOOLS)

for lecture_index in questions:
    # print(f"lecture_content: {lectures[lecture_index].content}")
    print(f"lecture_questions:")
    for question in questions[lecture_index].questions:
        print(question)

        messages = [
            {'role': 'system', 'content': 'You are given the task of evaluating examination questions given the lecture content and question.'},
            {'role': 'user', 'content': f'Lecture content: {lectures[lecture_index].content}\nQuestion: {question}'}
        ]

        evaluation = client.chat.completions.create(
            model=MODEL,
            response_model=Evaluation,
            messages=messages,
            temperature=0.3,
        )

        print(evaluation)
    

In [156]:
def extract_tag_content(text):
    # Define the regex pattern
    pattern = r'<(?P<tag>\w+)>(?P<content>.*?)</(?P=tag)>'
    
    # Find all matches in the text
    matches = re.finditer(pattern, text, re.DOTALL)
    
    # Extract and print the content
    result = {}
    for match in matches:
        tag = match.group('tag')
        content = match.group('content')
        if tag in result:
            result[tag].append(content)
        else:
            result[tag] = [content]
    
    return result

In [161]:
import tqdm 

MODEL = "bartowski/Mistral-7B-Instruct-v0.3-GGUF"
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

questions[lecture_index].evaluations = []

for lecture_index in questions:
    # print(f"lecture_content: {lectures[lecture_index].content}")
    print(f"lecture_questions:")
    for question in tqdm.tqdm(questions[lecture_index].questions):
        print(question)

        messages = [
            {'role': 'system', 'content': 'You are given the task of evaluating examination question given the lecture content within <lecture> </lecture> and question within <question> </question> tags. Provide a response in the following format: <reasoning>explain your evaluation in detail, including the section of the lecture that the question covers and your reasoning for the evaluation in markdown text</reasoning>\n <relevance>an integer from 0 to 10,  where 0 means irrelevant, 5 is still slightly bad, and 10 means a very important and relevant question; only respond with a single number</relevance>\n <difficulty>an integer from 0 to 10, where 10 is very very hard 5, 5 is average, and 0 is a silly question, in context of the student having taken the lecture already; only respond with a single number</difficulty>\n <answer>answer the given question in length. If the question has choices, instead reply with only the correct choice, otherwise, reply in length in textual form, explaining your reasoning for the answer in markdown text</answer>'},
            {'role': 'user', 'content': f'<lecture>{lectures[lecture_index].content}</lecture>; <question>{question}</question>'}
        ]

        completion = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            temperature=0.3,
        )

        # print(completion.choices[0].message.content)

        evaluation = extract_tag_content(completion.choices[0].message.content)
        print(f"evaluation: {evaluation}\naxes: {len(evaluation)}\n-----")

        questions[lecture_index].evaluations.append(evaluation)

lecture_questions:


  0%|          | 0/51 [00:00<?, ?it/s]

What is a chat bot and how does it integrate with various platforms like IRC, Discord, or social media? Provide an example of a general-purpose chatbot system.
A) Siri
B) Alexa
C) ChatGPT
D) A rule-based chatbot for customer support on a website



  0%|          | 0/51 [00:10<?, ?it/s]


KeyboardInterrupt: 

In [149]:
for lecture_index in questions:
    for index, question in enumerate(questions[lecture_index].questions):
        print(questions[lecture_index].questions[index])
        print(questions[lecture_index].evaluations[index])
        print("-----")

What is a chat bot and how does it integrate with various platforms like IRC, Discord, or social media? Provide an example of a general-purpose chatbot system.
A) Siri
B) Alexa
C) ChatGPT
D) A rule-based chatbot for customer support on a website

{'reason': "The question asks about the definition and integration of chat bots, which is covered in the lecture content under the section 'Chat Bots'. The question also requests an example of a general-purpose chatbot system. In this context, the correct answer is C) ChatGPT.", 'difficulty': '5', 'relevance': '10', 'answer': 'C) ChatGPT'}
-----
Which NLP tasks are commonly used in text generation, image generation from text (T2I), and question answering? List at least three tasks for each category.

{'reason': 'The question asks about the definition and integration of chat bots, their examples, and a general-purpose chatbot system. The lecture content covers various aspects of NLP, including text generation, question answering, and chatbots. 